## Uwekaji wa Vifurushi

Katika mfano wetu wa awali, tulifanya kazi na vekta za maneno za hali ya juu zenye urefu wa `vocab_size`, na tulikuwa tunabadilisha wazi kutoka kwa vekta za uwakilishi wa nafasi za hali ya chini kwenda kwenye uwakilishi wa sparse wa one-hot. Uwiano huu wa one-hot si wa ufanisi wa kumbukumbu, na zaidi ya hayo, kila neno linachukuliwa kuwa huru kutoka kwa mengine, yaani, vekta za one-hot hazionyeshi uhusiano wowote wa maana kati ya maneno.

Katika sehemu hii, tutaendelea kuchunguza seti ya data ya **News AG**. Kuanza, hebu tupakie data na tupate baadhi ya ufafanuzi kutoka kwenye daftari la awali.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Je, nini maana ya embedding?

Wazo la **embedding** ni kuwakilisha maneno kwa kutumia vekta zenye vipimo vya chini, ambazo kwa namna fulani zinaonyesha maana ya kisemantiki ya neno. Tutajadili baadaye jinsi ya kujenga embeddings za maneno zenye maana, lakini kwa sasa wacha tuzingatie embedding kama njia ya kupunguza vipimo vya vekta ya neno.

Kwa hivyo, safu ya embedding itachukua neno kama ingizo, na kutoa vekta ya matokeo yenye `embedding_size` maalum. Kwa namna fulani, hii ni sawa na safu ya `Linear`, lakini badala ya kuchukua vekta iliyosimbwa kwa one-hot, itakuwa na uwezo wa kuchukua namba ya neno kama ingizo.

Kwa kutumia safu ya embedding kama safu ya kwanza katika mtandao wetu, tunaweza kubadilisha kutoka mfuko wa maneno (bag-of-words) kwenda kwenye mfano wa **embedding bag**, ambapo tunabadilisha kila neno katika maandishi yetu kuwa embedding inayolingana, kisha tunahesabu kazi fulani ya jumla juu ya embeddings zote hizo, kama vile `sum`, `average` au `max`.

![Picha inayoonyesha classifier ya embedding kwa maneno matano ya mfululizo.](../../../../../translated_images/sw/embedding-classifier-example.b77f021a7ee67eee.webp)

Mtandao wetu wa neva wa kuainisha utaanza na safu ya embedding, kisha safu ya jumlisho, na classifier ya linear juu yake:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Kushughulikia Ukubwa wa Mfuatano wa Vigezo

Kutokana na usanifu huu, minibatches kwa mtandao wetu zitahitaji kuundwa kwa njia maalum. Katika sehemu iliyopita, tulipotumia bag-of-words, tensor zote za BoW katika minibatch zilikuwa na ukubwa sawa `vocab_size`, bila kujali urefu halisi wa mfuatano wa maandishi yetu. Mara tu tunapohamia kwenye word embeddings, tutakuwa na idadi tofauti ya maneno katika kila sampuli ya maandishi, na tunapochanganya sampuli hizo katika minibatches tutalazimika kutumia padding fulani.

Hili linaweza kufanyika kwa kutumia mbinu ile ile ya kutoa kazi ya `collate_fn` kwa chanzo cha data:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Kufundisha classifier ya embedding

Sasa kwa kuwa tumefafanua dataloader sahihi, tunaweza kufundisha modeli kwa kutumia kazi ya mafunzo tuliyofafanua katika kitengo kilichopita:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Kumbuka**: Tunafanya mafunzo kwa rekodi 25k tu hapa (chini ya kipindi kimoja kamili) kwa sababu ya muda, lakini unaweza kuendelea kufundisha, andika kazi ya kufundisha kwa vipindi kadhaa, na jaribu na kigezo cha kiwango cha kujifunza ili kufikia usahihi wa juu. Unapaswa kuwa na uwezo wa kufikia usahihi wa takriban 90%.


### Tabaka la EmbeddingBag na Uwakilishi wa Mfuatano wa Urefu Tofauti

Katika usanifu wa awali, tulihitaji kuongeza urefu wa mfuatano wote ili kufanana na urefu mmoja kwa ajili ya kuingiza kwenye kundi dogo (minibatch). Hii si njia bora zaidi ya kuwakilisha mfuatano wa urefu tofauti - njia nyingine inaweza kuwa kutumia **offset** vector, ambayo itahifadhi nafasi za mfuatano wote uliowekwa kwenye vector moja kubwa.

![Picha inayoonyesha uwakilishi wa mfuatano wa offset](../../../../../translated_images/sw/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Katika picha hapo juu, tunaonyesha mfuatano wa herufi, lakini katika mfano wetu tunafanya kazi na mfuatano wa maneno. Hata hivyo, kanuni ya jumla ya kuwakilisha mfuatano kwa kutumia offset vector inabaki ile ile.

Ili kufanya kazi na uwakilishi wa offset, tunatumia tabaka [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Tabaka hili linafanana na `Embedding`, lakini linachukua vector ya maudhui na vector ya offset kama pembejeo, na pia linajumuisha tabaka ya wastani, ambayo inaweza kuwa `mean`, `sum` au `max`.

Hapa kuna mtandao uliorekebishwa unaotumia `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Ili kuandaa seti ya data kwa mafunzo, tunahitaji kutoa kazi ya ubadilishaji ambayo itaandaa vector ya upendeleo:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Kumbuka, kwamba tofauti na mifano yote ya awali, mtandao wetu sasa unakubali vigezo viwili: vector ya data na vector ya offset, ambazo zina ukubwa tofauti. Vivyo hivyo, kipakiaji chetu cha data pia kinatupatia thamani 3 badala ya 2: vector za maandishi na offset zinatolewa kama vipengele. Kwa hivyo, tunahitaji kurekebisha kidogo kazi yetu ya mafunzo ili kushughulikia hilo:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Uwakilishi wa Semantiki: Word2Vec

Katika mfano wetu wa awali, safu ya kuingiza ya modeli ilijifunza kuwakilisha maneno kwa njia ya vekta, hata hivyo, uwakilishi huu haukuwa na maana ya kisemantiki sana. Ingekuwa vizuri kujifunza uwakilishi wa vekta ambapo maneno yanayofanana au visawe vinahusiana na vekta zilizo karibu kwa mujibu wa umbali fulani wa vekta (mfano, umbali wa euclidian).

Ili kufanikisha hilo, tunahitaji kufundisha modeli yetu ya kuingiza kwa maandishi mengi kwa njia maalum. Mojawapo ya njia za kwanza za kufundisha uwakilishi wa semantiki inaitwa [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Inategemea usanifu kuu mbili zinazotumika kuzalisha uwakilishi wa maneno ulioenea:

 - **Mfuko endelevu wa maneno** (CBoW) — katika usanifu huu, tunafundisha modeli kutabiri neno kutoka muktadha wa maneno yanayozunguka. Kwa kuzingatia ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, lengo la modeli ni kutabiri $W_0$ kutoka $(W_{-2},W_{-1},W_1,W_2)$.
 - **Skip-gram endelevu** ni kinyume cha CBoW. Modeli hutumia dirisha la muktadha wa maneno yanayozunguka kutabiri neno la sasa.

CBoW ni ya haraka, wakati skip-gram ni ya polepole, lakini inafanya kazi bora ya kuwakilisha maneno yasiyo ya kawaida.

![Picha inayoonyesha algorithimu za CBoW na Skip-Gram za kubadilisha maneno kuwa vekta.](../../../../../translated_images/sw/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Ili kujaribu kuingiza kwa Word2Vec iliyofundishwa awali kwenye seti ya data ya Google News, tunaweza kutumia maktaba ya **gensim**. Hapa chini tunapata maneno yanayofanana zaidi na 'neural'

> **Note:** Unapounda vekta za maneno kwa mara ya kwanza, kuzidownload kunaweza kuchukua muda!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Tunaweza pia kuhesabu embeddings za vector kutoka kwa neno, zitakazotumika katika kufundisha modeli ya uainishaji (tunaonyesha tu sehemu 20 za kwanza za vector kwa uwazi):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Jambo kubwa kuhusu upachikaji wa kisemantiki ni kwamba unaweza kudhibiti usimbaji wa vekta kubadilisha maana. Kwa mfano, tunaweza kuomba kutafuta neno, ambalo uwakilishi wake wa vekta ungekuwa karibu iwezekanavyo na maneno *mfalme* na *mwanamke*, na mbali na neno *mwanaume*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Mbinu za CBoW na Skip-Grams ni "utabiri" wa embeddings, kwa kuwa zinazingatia tu muktadha wa karibu. Word2Vec haichukui faida ya muktadha wa jumla.

**FastText**, inajengwa juu ya Word2Vec kwa kujifunza uwakilishi wa vekta kwa kila neno na n-grams za herufi zinazopatikana ndani ya kila neno. Thamani za uwakilishi zinajumlishwa kuwa vekta moja katika kila hatua ya mafunzo. Ingawa hii inaongeza hesabu nyingi za ziada wakati wa mafunzo ya awali, inaruhusu embeddings za maneno kuhifadhi taarifa za sehemu za maneno.

Njia nyingine, **GloVe**, inatumia wazo la matriki ya ushirikiano, ikitumia mbinu za neva kugawa matriki ya ushirikiano kuwa vekta za maneno zenye maelezo zaidi na zisizo za mstari.

Unaweza kucheza na mfano kwa kubadilisha embeddings kuwa FastText na GloVe, kwa kuwa gensim inaunga mkono mifano mbalimbali ya embeddings za maneno.


## Kutumia Embeddings Zilizofunzwa Awali katika PyTorch

Tunaweza kurekebisha mfano hapo juu ili kujaza awali matriki katika safu yetu ya embedding kwa kutumia embeddings za maana, kama Word2Vec. Tunapaswa kuzingatia kwamba msamiati wa embeddings zilizofunzwa awali na msamiati wa maandishi yetu huenda usifanane, kwa hivyo tutaweka uzito wa maneno yanayokosekana kwa thamani za nasibu:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Sasa hebu tufunze modeli yetu. Kumbuka kwamba muda unaochukua kufundisha modeli ni mkubwa zaidi kuliko katika mfano wa awali, kutokana na ukubwa mkubwa wa safu ya embedding, na hivyo idadi kubwa zaidi ya vigezo. Pia, kwa sababu ya hili, huenda tukahitaji kufundisha modeli yetu kwa mifano zaidi ikiwa tunataka kuepuka overfitting.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Katika hali yetu hatuoni ongezeko kubwa la usahihi, jambo ambalo huenda linatokana na tofauti kubwa za msamiati.  
Ili kushinda tatizo la tofauti za msamiati, tunaweza kutumia mojawapo ya suluhisho zifuatazo:  
* Kufunza upya modeli ya word2vec kwa kutumia msamiati wetu  
* Kupakia seti yetu ya data kwa kutumia msamiati kutoka kwenye modeli ya word2vec iliyofunzwa tayari. Msamiati unaotumika kupakia seti ya data unaweza kubainishwa wakati wa kupakia.  

Njia ya pili inaonekana rahisi zaidi, hasa kwa sababu mfumo wa PyTorch `torchtext` una msaada wa kujengwa ndani kwa embeddings. Kwa mfano, tunaweza kuanzisha msamiati unaotegemea GloVe kwa njia ifuatayo:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Msamiati uliojazwa una shughuli zifuatazo za msingi:  
* Kamusi ya `vocab.stoi` inatuwezesha kubadilisha neno kuwa faharasa yake ya kamusi.  
* `vocab.itos` hufanya kinyume chake - hubadilisha namba kuwa neno.  
* `vocab.vectors` ni safu ya vekta za uwekaji, kwa hivyo ili kupata uwekaji wa neno `s` tunahitaji kutumia `vocab.vectors[vocab.stoi[s]]`.  

Hapa kuna mfano wa kudhibiti uwekaji ili kuonyesha usawa **kind-man+woman = queen** (nililazimika kurekebisha kidogo mgawo ili ifanye kazi):  


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Ili kufundisha classifier kwa kutumia embeddings hizo, tunahitaji kwanza kusimba seti yetu ya data kwa kutumia msamiati wa GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Kama tulivyoona hapo juu, viambatisho vyote vya vector vinahifadhiwa katika matriki ya `vocab.vectors`. Hii inafanya iwe rahisi sana kupakia uzito huo kwenye uzito wa safu ya viambatisho kwa kutumia kunakili rahisi:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Moja ya sababu hatuoni ongezeko kubwa la usahihi ni kwa sababu baadhi ya maneno kutoka kwenye seti yetu ya data yanakosekana katika msamiati wa GloVe uliotangulizwa, na hivyo yanapuuzwa kimsingi. Ili kushinda hali hii, tunaweza kufundisha embeddings zetu wenyewe kwenye seti yetu ya data.


## Uwakilishi wa Muktadha wa Maneno

Kikwazo kimoja kikubwa cha uwakilishi wa jadi wa embeddings zilizofunzwa awali kama Word2Vec ni tatizo la kutofautisha maana ya maneno kulingana na muktadha. Ingawa embeddings zilizofunzwa awali zinaweza kunasa baadhi ya maana ya maneno katika muktadha, kila maana inayowezekana ya neno huwakilishwa katika embedding moja. Hili linaweza kusababisha changamoto katika mifano inayofuata, kwa kuwa maneno mengi kama 'play' yana maana tofauti kulingana na muktadha yanapotumika.

Kwa mfano, neno 'play' katika sentensi hizi mbili lina maana tofauti kabisa:
- Nilikwenda kwenye **play** ukumbini.
- John anataka **play** na marafiki zake.

Embeddings zilizofunzwa awali hapo juu zinawakilisha maana zote mbili za neno 'play' katika embedding moja. Ili kushinda kikwazo hiki, tunahitaji kujenga embeddings kulingana na **mfano wa lugha**, ambao umefunzwa kwenye mkusanyiko mkubwa wa maandishi, na *unajua* jinsi maneno yanavyoweza kuunganishwa katika muktadha tofauti. Kujadili uwakilishi wa muktadha wa maneno ni nje ya mada ya mafunzo haya, lakini tutarudi kwenye mada hii tutakapozungumzia mifano ya lugha katika sehemu inayofuata.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati asilia katika lugha yake ya awali inapaswa kuchukuliwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
